
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio A/B Testing Parametrico


#### Ejercicio 1:  Linea de montaje 

En una fabrica de componenetes textil se quiere comparar el tiempo de fabricacion de un determinado producto mediante tres métodos diferentes.  Se seleccionan aleatoriamente productos elaborados con cada uno de los métodos y se observa el tiempo de fabricación. 


1. ¿Se verifican los supuestos del modelo de análisis de la varianza?

2. ¿Existe evidencia de que el método influye en el tiempo medio de elaboración del producto?

3. La empresa valora que el tiempo de fabricación sea el menor posible. ¿Qué método cree que ofrece mejores resultados?

In [259]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st 
from scipy import stats  

from scipy.stats import skew,kurtosis # libreria para asimetria y curtosis 

### 1.Verificación de los supuestos del test ANOVA

In [260]:
df_linea_montaje=pd.read_csv('datos/linea_montaje.csv')

In [261]:
df_linea_montaje.head()



,minutos,metodo
0,4.166484,A
1,4.887466,A
2,0.727608,A
3,8.280542,A
4,1.413129,A


In [262]:
df_linea_montaje.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   minutos  150 non-null    float64
 1   metodo   150 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.5+ KB


El tamaño de la muestra es 150, aunque dividida en los tres grupos de metodos es 50. Como el tamaño es mayor que 30 tenemos que aplicamos el kstest


In [263]:
df_linea_montaje.describe()

,minutos
count,150.000000
mean,4.895103
std,2.222707
min,-0.818899
25%,3.528356
50%,4.870153
75%,6.328776
max,12.059491


Vemos que el minimo de los valore que toma la columna minutos es un valor negativo. A la hora del kstest no afectan al calculo, así que los dejamos

In [264]:
# Definimos una funcion que calcula el test ks en cada grupo, devolvera el metodo y una tupla en la que el valor de
# la derecha es el p-valor, que es lo que nos interesa en este caso
def test_ks(grupo):
    mean, std = grupo.mean(), grupo.std()
    return stats.kstest(grupo, 'norm', args=(mean, std))

normalidad_ks_resultados = df_linea_montaje.groupby('metodo')['minutos'].apply(test_ks)

print(normalidad_ks_resultados)


metodo
A     (0.08510025236028507, 0.831915919566997)
B     (0.1126595913827656, 0.5135125149364379)
C    (0.10224330731103809, 0.6354526500316278)
Name: minutos, dtype: object


El p-valor es mayor que 0,05 en todos los grupos luego se cumple la condición de normalidad. Veamos la homoestaticidad

In [265]:
grupos = df_linea_montaje.groupby('metodo')['minutos'].apply(list)

# Aplicamos el test de Bartlett usando los grupos generados por el groupby
statistic, p_valor = stats.bartlett(*grupos)

print(f"statistic = {statistic}, p_valor = {p_valor}")


statistic = 0.7587362059281331, p_valor = 0.6842936757694211


El p-valor es mayor que 0,05 luego asumimos la homoestaticidad

Las mediciones de tiempos en las que consiste el experimento del enunciado son independientes, pues ninguna depende de otra, por lo que también está comprobada la independencia. Se cumplen las condiciones del test ANOVA

### 2. ¿Existe evidencia de que el método influye en el tiempo medio de elaboración del producto?

Si tomamos como hipotesis nula el que todos los metodos tienen el mismo tiempo de elaboración, aplicamos un ANOVA y si el p-valor nos diera por debajo de 0,05 tendríamso evidencia estadísicatica de que al menos uno de los metodos tiene un tiempo  de elaboración diferente y por tanto se debe rechazar la hipotesis nula

In [266]:
anova_statistic, p_valor = stats.f_oneway(*grupos)

anova_statistic, p_valor, 

(np.float64(4.275128289148849), np.float64(0.015679251126397998))

El p-valor es 0,016<0,05. Luego se rechaza la hipotesis nula y por tanto hay al enos un metodo que tiene un tiempo de elaboracion diferente

### 3. La empresa valora que el tiempo de fabricación sea el menor posible. ¿Qué método cree que ofrece mejores resultados?

In [267]:
# Calculamos la media de minutos para cada método
media_por_metodo = df_linea_montaje.groupby('metodo')['minutos'].mean()
media_por_metodo

metodo
A    4.720512
B    4.364523
C    5.600274
Name: minutos, dtype: float64

A la vista de los calculos anteriores, el metodo B es el que ofrece mejroes resultados

#### Ejercicio 2: Densidad de Fluidos 


La convección es un mecanismo fundamental de transferencia de calor en fluidos, que ocurre debido a las variaciones en la densidad provocadas por diferencias de temperatura. Este fenómeno se manifiesta cuando las porciones del fluido que se calientan, al volverse menos densas, ascienden hacia regiones de menor temperatura, mientras que las porciones más frías, al ser más densas, descienden hacia las zonas donde la temperatura es más alta. Este movimiento continuo da lugar a las llamadas corrientes de convección, las cuales desempeñan un papel crucial en la distribución homogénea de la temperatura dentro del fluido, promoviendo un equilibrio térmico.

En este contexto, se ha diseñado y ejecutado un experimento con el propósito de investigar cómo se ve afectada la densidad de un fluido cuando se incrementa la temperatura en una región específica del sistema; por tanto se asegura la independencia y aleatoriedad del mismo. El objetivo del experimento es comprender en detalle las modificaciones en la densidad que resultan de este aumento térmico y cómo estas afectan el comportamiento global del fluido.


1. Analiza las suposiciones del modelo: ¿Se cumplen las condiciones de homocedasticidad, independencia y normalidad?
2. ¿Tiene la temperatura un impacto notable en la densidad del fluido?
3. ¿Es posible concluir que las temperaturas de 100 y 125 provocan, en promedio, una menor densidad en el fluido comparadas con las temperaturas de 150 y 175?

### 1. Analiza las suposiciones del modelo: ¿Se cumplen las condiciones de homocedasticidad, independencia y normalidad?


In [268]:
df_densidad_fluidos=pd.read_csv('datos/densidad_fluidos.csv')

In [269]:
df_densidad_fluidos.head()

,temperatura,densidad
0,100,21.790655
1,100,21.725900
2,100,21.806052
3,100,21.895319
4,100,21.716121


In [270]:
df_densidad_fluidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temperatura  400 non-null    int64  
 1   densidad     400 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.4 KB


In [271]:
df_densidad_fluidos.describe()

,temperatura,densidad
count,400.000000,400.000000
mean,137.500000,21.733028
std,27.985854,0.169214
min,100.000000,21.323342
25%,118.750000,21.614630
50%,137.500000,21.742521
75%,156.250000,21.843457
max,175.000000,22.170432


Tenemos una muestra de tamaño 400 dividida en grupos, serán dos de tamaño 200, empleamos pues kstest para estudir la normalidad

In [272]:
#Empleamos de nuevo la función que definimos paa el primer ejercicio
def test_ks(grupo):
    mean, std = grupo.mean(), grupo.std()
    return stats.kstest(grupo, 'norm', args=(mean, std))

normalidad_ks_resultados = df_densidad_fluidos.groupby('temperatura')['densidad'].apply(test_ks)

print(normalidad_ks_resultados)




temperatura
100    (0.051776473605968876, 0.9386471491697437)
125     (0.07210369396849026, 0.6491556422553408)
150     (0.05555454410249083, 0.9000738973191298)
175    (0.052646760818004956, 0.9306618656035937)
Name: densidad, dtype: object


El p valor, reflejado en la columna de la derecha del output anterior es muy alto para todos los grupos de temperatura, mayor que 0,05. Por lo que se satisface la condición de normalidad

Para la homoestacidad, dada la alta normalida de los datos, empleamos el test de Bartlet

In [273]:
grupos_temperatura = df_densidad_fluidos.groupby('temperatura')['densidad'].apply(list)


In [274]:
statistic, p_valor = stats.bartlett(*grupos_temperatura)

print(f"statistic = {statistic}, p_valor = {p_valor}")

statistic = 4.680218828816711, p_valor = 0.19676755827086465


el p_valor es 0,20 redondeando, superior a 0,05 luego se satisface la condición de homoestacidad.

La independencia del experimento viene asegurada en el propio enunciado así que se satisfacen las 3 condiciones.

2. ¿Tiene la temperatura un impacto notable en la densidad del fluido?


Tomamos como hipotesis nula el que la densidad del fluido no dependa de la temperatura, aplicamos un ANOVA y si el p-valor nos da por debajo de 0,05 rechazamos la hipotesis nula.

In [275]:
resultado_anova = stats.f_oneway(*grupos_temperatura)
resultado_anova

F_onewayResult(statistic=np.float64(212.84277610615817), pvalue=np.float64(3.327798065119595e-82))

el p-valor es una potencia de exponete -82. Es decir, un valor pequeñisimo. Por lo que efectivamente tenemos un impacto notable de la temperatura sobre la densidad

### 3. ¿Es posible concluir que las temperaturas de 100 y 125 provocan, en promedio, una menor densidad en el fluido comparadas con las temperaturas de 150 y 175

Vamos a hacer un test de t de student.

In [276]:
grupo_bajo = df_densidad_fluidos[df_densidad_fluidos['temperatura'].isin([100, 125])]['densidad']
grupo_alto = df_densidad_fluidos[df_densidad_fluidos['temperatura'].isin([150, 175])]['densidad']
prueba_t = stats.ttest_ind(grupo_bajo, grupo_alto, equal_var=True)
prueba_t

TtestResult(statistic=np.float64(-15.129677894969593), pvalue=np.float64(3.563849634924047e-41), df=np.float64(398.0))

El test t de Student de comparación de medias entre los grupos (100 y 125) y (150 y 175) muestra un valor t de aproximadamente -15.13 y un p-valor bajísimo (3.56e-41). Esto sugiere que existe una diferencia estadísticamente significativa entre estos grupos, con las temperaturas de 100 y 125 asociadas, en promedio, a una menor densidad en comparación con las de 150 y 175.